# Finite Tree Automata for Domain Specific Languages

Problem: manipulating trees representing expressions in a DSL:
![tree more or less](./hpsi.svg "hpsi BigDFT")
Finding patterns that can be factored/transformed into optimized operators.

Maybe using an equivalent of regular expressions on trees?

Reference : TATA (Tree Automata Technique and Applications)

## I. Playing with Trees

In [ ]:
require_relative '../lib/libtree'
require_relative '../lib/libtree/graphviz'

### Defining a System and a Tree

In [ ]:
mod = LibTree::define_system( 
  alphabet: {
    f: 3,
    g: 2,
    h: 1,
    a: 0,
    b: 0
  },
  variables: [:x, :y]
)
extend mod
t = f(g(a,b),a,h(b))
t.to_s

In [ ]:
IRuby::display(t.to_graphviz.output( png: String ), mime: "image/png" )
nil;

### Positions

In [ ]:
p t.positions
p t[0,1]
nil;

In [ ]:
t[0,1]= g(a,a)
IRuby::display(t.to_graphviz.output( png: String ), mime: "image/png" )
nil;

### Iterations

In [ ]:
t.each { |n| print n.symbol }
puts
t.each(:post) { |n| print n.symbol }
nil;

In [ ]:
t.each(:pre) { |n| print n.symbol }
nil;

### Substitutions

In [ ]:
t = f(x,x,y)
IRuby::display(t.to_graphviz.output( png: String ), mime: "image/png" )
nil;

In [ ]:
s = substitution( rules: { 
  x => a,
  y => g(b, b)
  } )
t2 = s[t]
IRuby::display(t2.to_graphviz.output( png: String ), mime: "image/png" )
nil;

## Homomorphism

In [ ]:
mod = LibTree::define_system( alphabet: {h: 3, g: 3, f: 2, a: 0, b: 0} )
extend mod
homomorphism = LibTree::Homomorphism::new(
  input_system: mod,
  output_system: mod,
  variables: [x1, x2, x3],
  rules: { 
    :g => f(x1, f(x2, x3))
    }
  )
t = g(a,g(b,b,b),a)
nil;

In [ ]:
IRuby::display(t.to_graphviz.output( png: String ), mime: "image/png" )
nil;

In [ ]:
IRuby::display(homomorphism[t].to_graphviz.output( png: String ), mime: "image/png" )
nil;

## II. Automata

### Bottom Up Automaton

In [ ]:
mod = LibTree::define_system( 
  alphabet: {
    o: 2,
    a: 2,
    n: 1,
    one: 0,
    zero: 0
    }
  )
puts mod
extend mod
nil;

In [ ]:
t1 = a(n(o(zero,one)),o(one,n(zero)))
t2 = a(n(n(o(zero,one))),o(one,n(zero)))
IRuby::display(t1.to_graphviz.output( png: String ), mime: "image/png" )
IRuby::display(t2.to_graphviz.output( png: String ), mime: "image/png" )
nil;

In [ ]:
automaton = LibTree::Automaton::new(
  system: mod,
  states: [:q0, :q1],
  final_states: [:q1],
  rules: {
    zero => :q0,
    one => :q1,
    n(:q0) => :q1,
    n(:q1) => :q0,
    a(:q0, :q0) => :q0,
    a(:q1, :q0) => :q0,
    a(:q0, :q1) => :q0,
    a(:q1, :q1) => :q1,
    o(:q0, :q0) => :q0,
    o(:q1, :q0) => :q1,
    o(:q0, :q1) => :q1,
    o(:q1, :q1) => :q1
    }
  )
puts automaton
nil;

In [ ]:
r = automaton.run(t1, rewrite: false)
IRuby::display(r.tree.to_graphviz.output( png: String ), mime: "image/png" )
sleep 3

loop do
  r.move
  IRuby::Display::clear_output(true)
  IRuby::display(r.tree.to_graphviz.output( png: String ), mime: "image/png" )
  sleep 1
end
puts "Sucessfull? #{r.successful?}"
nil;

In [ ]:
r = automaton.run(t2, rewrite: false)
IRuby::display(r.tree.to_graphviz.output( png: String ), mime: "image/png" )
sleep 3
loop do
  r.move
  IRuby::Display::clear_output(true)
  IRuby::display(r.tree.to_graphviz.output( png: String ), mime: "image/png" )
  sleep 1
end
puts "Sucessfull? #{r.successful?}"
nil;

### Rewriting Automaton

In [ ]:
r = automaton.run(t2)
IRuby::display(r.tree.to_graphviz.output( png: String ), mime: "image/png" )
sleep 3
loop do
  r.move
  IRuby::Display::clear_output(true)
  IRuby::display(r.tree.to_graphviz.output( png: String ), mime: "image/png" )
  sleep 1
end
puts "Sucessfull? #{r.successful?}"
nil;

### Top Down Automaton

In [ ]:
mod = LibTree::define_system( 
  alphabet: {
    one: 1, 
    zero: 1, 
    nill: 0
    },
  variables: [
    :x,
    ],
  states: [
    :q0,
    :q1,
    :q2
    ]
  )
extend mod
nil;

In [ ]:
t1 = one(one(zero(nill)))
t2 = one(zero(nill))
IRuby::display(t1.to_graphviz.output( png: String ), mime: "image/png" )
IRuby::display(t2.to_graphviz.output( png: String ), mime: "image/png" )
nil;

In [ ]:
automaton = LibTree::TopDownAutomaton::new(
  system: mod,
  states: [:q0, :q1, :q2],
  initial_states: [:q0],
  rules: {
    q0(nill) => nill,
    q0(zero(x)) => zero(q0()),
    q0( one(x)) =>  one(q1()),
    q1(zero(x)) => zero(q2()),
    q1( one(x)) =>  one(q0()),
    q2(zero(x)) => zero(q1()),
    q2( one(x)) =>  one(q2())
    }
  )
puts automaton
nil;

In [ ]:
r = automaton.run(t1)
IRuby::display(r.tree.to_graphviz.output( png: String ), mime: "image/png" )
sleep 3

loop do
  r.move
  IRuby::Display::clear_output(true)
  IRuby::display(r.tree.to_graphviz.output( png: String ), mime: "image/png" )
  sleep 1
end
puts "Sucessfull? #{r.successful?}"
nil;

In [ ]:
r = automaton.run(t2, rewrite: false)
IRuby::display(r.tree.to_graphviz.output( png: String ), mime: "image/png" )
sleep 3
loop do
  r.move
  IRuby::Display::clear_output(true)
  IRuby::display(r.tree.to_graphviz.output( png: String ), mime: "image/png" )
  sleep 1
end
puts "Sucessfull? #{r.successful?}"
nil;

In [ ]:
r = automaton.to_bottom_up_automaton.run(t1, rewrite: false)
IRuby::display(r.tree.to_graphviz.output( png: String ), mime: "image/png" )
sleep 3

loop do
  r.move
  IRuby::Display::clear_output(true)
  IRuby::display(r.tree.to_graphviz.output( png: String ), mime: "image/png" )
  sleep 1
end
puts "Sucessfull? #{r.successful?}"
nil;

In [ ]:
puts automaton.to_bottom_up_automaton

## Algorithms on Automata

List of non empty natural integer automaton.

In [ ]:
mod = LibTree::define_system( 
  alphabet: {
    cons: 2,
    s: 1,
    zero: 0,
    empt: 0
    },
  variables: []
  )
extend mod
automaton = LibTree::Automaton::new(
  system: mod, 
  states: [:qnat, :qlist, :qnelist, :qminusone],
  final_states: [:qnelist],
  rules: {
    zero => :qnat,
    s(:qnat) => :qnat,
    s(:qminusone) => :qnat,
    empt => :qlist,
    cons(:qnat, :qlist) => :qnelist,
    :qnelist => :qlist
    }
  )
puts automaton
nil;

### Epsilon Rules Removal
Epsilon transitive closure.

In [ ]:
automaton2 = automaton.remove_epsilon_rules
puts automaton2
nil;

### Reduction
Remove unaccessible states

In [ ]:
automaton3 = automaton2.reduce
puts automaton3
nil;

### Determinization

In [ ]:
automaton4 = automaton3.determinize
puts automaton4
nil;

In [ ]:
automaton4.rename_states
puts automaton4
nil;

In [ ]:
automaton4.rename_states( mapping: {:qr0 => :qnat, :qr1 => :qlist, :qr2 => :qnelist})
puts automaton4
nil

### Completion

In [ ]:
automaton5 = automaton4.complete
puts automaton5
nil

### Union

In [ ]:
automaton_prime = LibTree::Automaton::new(
  system: mod, 
  states: [:qnat, :qlist],
  final_states: [:qlist],
  rules: {
    zero => :qnat,
    s(:qnat) => :qnat,
    empt => :qlist,
    cons(:qnat, :qlist) => :qlist,
    }
  )
puts automaton_prime
nil;

In [ ]:
automaton_prime2 = automaton_prime.complete
puts automaton_prime2
nil;

In [ ]:
automaton6 = automaton5 | automaton_prime2
puts automaton6
nil;

### Minimization

In [ ]:
puts automaton6.minimize.rename_states

In [ ]:
puts automaton_prime2

## Grammars (Regular)

In [ ]:
non_terminals = LibTree::define_system( 
  alphabet: { 
    list: 0, 
    nat: 0
    }
  )
terminals = LibTree::define_system( 
  alphabet: {
    zero: 0,
    void: 0,
    s: 1,
    cons: 2
    }
  )
extend terminals
extend non_terminals
grammar = LibTree::RegularGrammar::new( 
  axiom: list, 
  non_terminals: non_terminals,
  terminals: terminals,
  rules: {
    list => [ void, cons(nat, list)],
    nat => [ zero, s(nat) ]
    }
  )
puts grammar
nil;

### Derivation

In [ ]:
puts grammar.derivation.derivation
puts grammar.derivation.derivation
puts grammar.derivation.derivation
puts grammar.derivation.derivation

### Equivalent Automaton...

In [ ]:
puts grammar.bottom_up_automaton
automaton = grammar.top_down_automaton
puts automaton

In [ ]:
4.times {
  r = automaton.run(grammar.derivation.derivation)
  r.run
  puts "Successful? #{r.successful?}"
  }
nil;

### ...And Back.

In [ ]:
puts automaton.to_grammar

# Now On to Regular Expressions!!!
Composition and concatenations of grammars.